# Library Imports

In [ ]:
# Required installs for your imports

%pip install opencv-python numpy tqdm matplotlib pyyaml

# for cv2
# for numpy
# for tqdm progress bar
# for plotting
# for yaml


In [ ]:
import os
import random
import cv2
import numpy as np
from IPython.display import display, Image
from tqdm import tqdm 
import random
import matplotlib.pyplot as plt
import yaml
import shutil

# Info and Steps

Dataset - datasetV3 - has all imgs and labels
steps:
1. upload dataset from local
2. these are full images - create 640x640 tiles and save 
3. use cv folds = 3
4. Sample dataset into
   4.1 70% train
   4.2 20% val
   4.3 10% test
   4.4 Train 70% of data using YOLO11n data
   4.5 Validate model on val data
   4.6 Test on test data
   4.7 store results
   4.5 increment cv_fold++
5. Select model with best metrics (should not underfit or overfit)

# Upload and Installs

1. Upload Dataset - waldo-dataset-v3

datasetV3

datasetV3/train_val
datasetV3/train_val/images
datasetV3/train_val/labels


datasetV3/test
datasetV3/test/images
datasetV3/test/labels

2. Install utralytics

In [ ]:
%pip install ultralytics
# and restart kernel

In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6411.4/8062.4 GB disk)


3. Install Scikit-learn, pandas, and PyYAML

In [ ]:
%pip install -U scikit-learn pandas pyyaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 87.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 98.8 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.2.2 requires pandas<2.2.4dev0

## Retrieve all label files for your dataset

In [12]:
from pathlib import Path

dataset_path = Path("/kaggle/working/datasetV3/train_val") # replace with 'path/to/dataset' for your custom data
#print(dataset_path.rglob("*labels/*.txt"))
labels = sorted(dataset_path.rglob("*labels/*.txt"))  # all data in 'labels'

print(labels[:3])

[PosixPath('/kaggle/working/datasetV3/train_val/labels/100_png.rf.ea7401b8d1f57435c8547882b54d7a05_0.txt'), PosixPath('/kaggle/working/datasetV3/train_val/labels/100_png.rf.ea7401b8d1f57435c8547882b54d7a05_1.txt'), PosixPath('/kaggle/working/datasetV3/train_val/labels/100_png.rf.ea7401b8d1f57435c8547882b54d7a05_2.txt')]


## Now, read the contents of the dataset YAML file and extract the indices of the class labels.



In [ ]:
import yaml

yaml_file = "./data.yaml"  # your data YAML with data directories and names dictionary

with open(yaml_file, encoding="utf8") as y:
    classes = yaml.safe_load(y)["names"]
# Create a dict {index: class_name}
cls_idx = {i: name for i, name in enumerate(classes)}

## Initialize an empty pandas DataFrame.



In [14]:
import pandas as pd

index = [label.stem for label in labels]  # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=index)

## Count the instances of each class-label present in the annotation files.



In [15]:
from collections import Counter

for label in labels:
    lbl_counter = Counter()

    with open(label) as lf:
        lines = lf.readlines()

    for line in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(line.split(" ", 1)[0])] += 1

    labels_df.loc[label.stem] = lbl_counter


labels_df = labels_df.fillna(0.0)  # replace `nan` values with `0.0`

#print(labels_df[290:310])

/tmp/ipykernel_36/1153986798.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels_df = labels_df.fillna(0.0)  # replace `nan` values with `0.0`


## K-Fold dataset split

## 1. Now we will use the KFold class from sklearn.model_selection to generate k splits of the dataset.

Important:
Setting shuffle=True ensures a randomized distribution of classes in your splits.
By setting random_state=M where M is a chosen integer, you can obtain repeatable results.

In [16]:
import random

from sklearn.model_selection import KFold

random.seed(0)  # for reproducibility
ksplit = 3
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)  # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

## 2. The dataset has now been split into k folds, each having a list of train and val indices. We will construct a DataFrame to display these results more clearly.

In [17]:
folds = [f"split_{n}" for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=index, columns=folds)

for i, (train, val) in enumerate(kfolds, start=1):
    folds_df[f"split_{i}"].loc[labels_df.iloc[train].index] = "train"
    folds_df[f"split_{i}"].loc[labels_df.iloc[val].index] = "val"

/tmp/ipykernel_36/3207785292.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  folds_df[f"split_{i}"].loc[labels_df.iloc[train].index] = "train"
/tmp/ipykernel_36/3207785292.py:6: FutureWarning: ChainedAssignmentError: behaviour will chang

## 3. Now we will calculate the distribution of class labels for each fold as a ratio of the classes present in val to those present in train.

In [18]:
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1e-7)
    fold_lbl_distrb.loc[f"split_{n}"] = ratio

## 4. Next, we create the directories and dataset YAML files for each split.

In [19]:
import datetime

supported_extensions = [".jpg", ".jpeg", ".png"]

# Initialize an empty list to store image file paths
images = []

# Loop through supported extensions and gather image files
for ext in supported_extensions:
    images.extend(sorted((dataset_path / "images").rglob(f"*{ext}")))

# Create the necessary directories and dataset YAML files
save_path = Path(dataset_path / f"{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val")
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "labels").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "labels").mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f"{split}_dataset.yaml"
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, "w") as ds_y:
        yaml.safe_dump(
            {
                "path": split_dir.as_posix(),
                "train": "train",
                "val": "val",
                "names": classes,
            },
            ds_y,
        )

## 5. Lastly, copy images and labels into the respective directory ('train' or 'val') for each split.

NOTE: The time required for this portion of the code will vary based on the size of your dataset and your system hardware.

In [20]:
import shutil

from tqdm import tqdm

for image, label in tqdm(zip(images, labels), total=len(images), desc="Copying files"):
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directory
        img_to_path = save_path / split / k_split / "images"
        lbl_to_path = save_path / split / k_split / "labels"

        # Copy image and label files to new directory (SamefileError if file already exists)
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(label, lbl_to_path / label.name)

Copying files: 100%|██████████| 459/459 [00:00<00:00, 873.00it/s]


## 6. save records

In [21]:
folds_df.to_csv(save_path / "kfold_datasplit.csv")
fold_lbl_distrb.to_csv(save_path / "kfold_label_distribution.csv")

## Load YOLO Model

In [13]:
from ultralytics import YOLO

model = (YOLO("yolo11n.yaml").load("yolo11n.pt"))

Transferred 499/499 items from pretrained weights


In [14]:
from ultralytics import YOLO

weights_path = "./yolo11n.pt"  # use yolo11n.pt for a small model
model = YOLO(weights_path, task="detect")

In [22]:
results = {}

# Define your additional arguments here
batch = 16
project = "kfold_demo"
epochs = 100

for k, dataset_yaml in enumerate(ds_yamls):
    model = YOLO(weights_path, task="detect")
    results[k] = model.train(
        data=dataset_yaml, epochs=epochs, batch=batch, project=project, name=f"fold_{k + 1}"
    )  # include any additional train arguments

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/datasetV3/train_val/2025-08-17_3-Fold_Cross-val/split_1/split_1_dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/working/yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=fold_1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, o

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/working/datasetV3/train_val/2025-08-17_3-Fold_Cross-val/split_1/train/labels... 306 images, 115 backgrounds, 1 corrupt: 100%|██████████| 306/306 [00:00<00:00, 1599.36it/s]

train: /kaggle/working/datasetV3/train_val/2025-08-17_3-Fold_Cross-val/split_1/train/images/16_jpg.rf.4d52687b758744d898985bfba3167576_6.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0187      1.0729]


train: New cache created: /kaggle/working/datasetV3/train_val/2025-08-17_3-Fold_Cross-val/split_1/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 958.5±757.5 MB/s, size: 169.6 KB)


...

    100/100       4.1G     0.6273     0.4604     0.8913          2        640: 100%|██████████| 20/20 [00:03<00:00,  5.94it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  6.17it/s]

                   all        152         96      0.947      0.948      0.971      0.795



100 epochs completed in 0.129 hours.
Optimizer stripped from kfold_demo/fold_3/weights/last.pt, 5.5MB
Optimizer stripped from kfold_demo/fold_3/weights/best.pt, 5.5MB

Validating kfold_demo/fold_3/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  2.67it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        152         96      0.938      0.942      0.976      0.799
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 4.7ms postprocess per image
Results saved to kfold_demo/fold_3


In [ ]:
model = YOLO("./kfold_demo/fold_1/weights/best.pt")  # load a custom model
dataYaml_split1 = "./datasetV3/2025-08-17_3-Fold_Cross-val/split_1/split_1_dataset.yaml"
# Validate the model
metrics = model.val(data=dataYaml_split1, imgsz=640, plots=True)  # no arguments needed, dataset and settings remembered
# mAP, precision, recall
# For single-class dataset
print(metrics.to_json())

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2069.1±254.9 MB/s, size: 157.1 KB)


val: Scanning /kaggle/working/datasetV3/train_val/2025-08-17_3-Fold_Cross-val/split_1/val/labels.cache... 153 images, 67 backgrounds, 0 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02<00:00,  4.60it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        153         91      0.941      0.871      0.959      0.805
Speed: 2.4ms preprocess, 4.8ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/val2
[
  {
    "Class":"waldo",
    "Images":86,
    "Instances":91,
    "Box-P":0.94065,
    "Box-R":0.87093,
    "Box-F1":0.90445,
    "mAP50":0.95864,
    "mAP50-95":0.80465
  }
]


In [ ]:
model = YOLO("./kfold_demo/fold_2/weights/best.pt")  # load a custom model
dataYaml_split2 = "./datasetV3/2025-08-17_3-Fold_Cross-val/split_2/split_2_dataset.yaml"
# Validate the model
metrics = model.val(data=dataYaml_split2, imgsz=640, plots=True)  # no arguments needed, dataset and settings remembered
# mAP, precision, recall
# For single-class dataset
print(metrics.to_json())

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2022.7±738.0 MB/s, size: 146.9 KB)


val: Scanning /kaggle/working/datasetV3/train_val/2025-08-17_3-Fold_Cross-val/split_2/val/labels.cache... 153 images, 59 backgrounds, 0 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02<00:00,  4.43it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        153         94      0.966      0.979      0.989      0.851
Speed: 2.2ms preprocess, 5.0ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/val3
[
  {
    "Class":"waldo",
    "Images":94,
    "Instances":94,
    "Box-P":0.96639,
    "Box-R":0.97872,
    "Box-F1":0.97252,
    "mAP50":0.98896,
    "mAP50-95":0.85139
  }
]


In [ ]:
model = YOLO("./kfold_demo/fold_3/weights/best.pt")  # load a custom model
dataYaml_split3 = "./datasetV3/2025-08-17_3-Fold_Cross-val/split_3/split_3_dataset.yaml"
# Validate the model
metrics = model.val(data=dataYaml_split3, imgsz=640, plots=True)  # no arguments needed, dataset and settings remembered
# mAP, precision, recall
# For single-class dataset
print(metrics.to_json())

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1793.1±792.9 MB/s, size: 147.1 KB)


val: Scanning /kaggle/working/datasetV3/train_val/2025-08-17_3-Fold_Cross-val/split_3/val/labels.cache... 153 images, 56 backgrounds, 1 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]

val: /kaggle/working/datasetV3/train_val/2025-08-17_3-Fold_Cross-val/split_3/val/images/16_jpg.rf.4d52687b758744d898985bfba3167576_6.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0187      1.0729]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02<00:00,  4.25it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        152         96      0.938      0.942      0.976      0.805
Speed: 2.0ms preprocess, 4.7ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/val4
[
  {
    "Class":"waldo",
    "Images":96,
    "Instances":96,
    "Box-P":0.93775,
    "Box-R":0.94154,
    "Box-F1":0.93964,
    "mAP50":0.97625,
    "mAP50-95":0.80532
  }
]


In [ ]:
os.makedirs('./waldo_test_split1')
os.makedirs('./waldo_test_split2')
os.makedirs('./waldo_test_split3')

In [38]:
import os
from PIL import Image, ImageDraw
import pandas as pd


def list_image_files(folder_path):
    all_files = os.listdir(folder_path)
    # filter out jpg and png files
    img_files = [
        file for file in all_files if file.lower().endswith((".jpg", ".jpeg", ".png"))
    ]
    # get files full path
    img_files = [os.path.join(folder_path, file) for file in img_files]
    return img_files


def get_box_coordonates(section_xy, section_wh, box_xywhn=None):
    section_width, section_height = section_wh
    section_x, section_y = section_xy
    # opened image size

    label_data = box_xywhn
    label_norm_x: int = label_data[1]  # box center x coordonate (NORMALISED)
    label_norm_y: int = label_data[2]  # box center y coordonate (NORMALISED)
    label_norm_box_width = label_data[3]  # box width (NORMALISED)
    label_norm_box_height = label_data[4]  # box height (NORMALISED)

    # box coordonates
    x0: int = int(label_norm_x * section_width)
    y0: int = int(label_norm_y * section_height)
    label_width: int = int(label_norm_box_width * section_width)
    label_height: int = int(label_norm_box_height * section_height)
    box = (
        int(section_x + (x0 - int(label_width / 2))),
        int(section_y + (y0 - int(label_height / 2))),
        int(section_x + (x0 + int(label_width / 2))),
        int(section_y + (y0 + int(label_height / 2))),
    )

    # breakpoint()

    return box


def load_label(label_path):
    df = pd.read_csv(filepath_or_buffer=label_path, header=None, sep=" ")
    return df.iloc[0]


def crop_image(image, xy=(50, 50), wh=(640, 640)):
    x, y = xy
    right, bottom = tuple(a + b for a, b in zip(wh, xy))
    image = image.crop((x, y, right, bottom))
    return image


# image path - 'relative or absolute path to the image we are loading
# size - Tuple (width, height) - the size of the box
def draw_rectangle(
    image_path=None, image=None, label_path=None, label_contents=None, return_image=True
):
    if image_path != None:
        image = Image.open(image_path)
        img_name, _ = os.path.splitext(os.path.basename(image_path))

    # opened image size
    img_width = image.width
    img_height = image.height
    label_data = ""
    box = None  # initialising box tuple
    if label_path != None:
        label_path = (
            label_path
            if label_path is not None
            else f"./datasets/train/labels/{img_name}.txt"
        )
        label_data = load_label(label_path)

        label_norm_x = label_data[1]
        label_norm_y = label_data[2]
        label_norm_box_width = label_data[3]
        label_norm_box_height = label_data[4]
        # box coordonates
        x0: int = int(label_norm_x * img_width)
        y0: int = int(label_norm_y * img_height)
        x1: int = int(label_norm_box_width * img_width)
        y1: int = int(label_norm_box_height * img_height)

        box = ((x0 - x1)*120 , (y0 - y1)*120, ((x0 + x1)*120 , (y0 + y1)*120 ))
    if label_contents != None:
        box = label_contents

    # load label data
    draw = ImageDraw.Draw(image)

    draw.rectangle(box, outline="blue", width=10)
    if return_image == True:
        return image
    else:
        image.show()


def create_files(image_content, label_content, image_path, label_path):
    # saving the image
    image_content.save(image_path)
    # saving the label
    with open(label_path) as file:
        file.write(label_content)


def get_labelIn_window(
    section_xy, section_wh, orig_label_xy, orig_label_wh, label_number=0
):
    # destructure parameter data from tuples into variables
    section_x, section_y = section_xy
    section_w, section_h = section_wh
    orig_label_x, orig_label_y = orig_label_xy
    orig_label_w, orig_label_h = orig_label_wh

    # get new label xy coordonates based on window position and original label position
    new_label_x = (orig_label_x - (orig_label_w / 2)) - section_x
    new_label_y = (orig_label_y - (orig_label_h / 2)) - section_y

    # set new label coordonates (normalised)
    norm_new_label_x = round(new_label_x / section_w, 6)
    norm_new_label_y = round(new_label_y / section_h, 6)
    norm_orig_label_w = round(orig_label_w / section_w, 6)
    norm_orig_label_h = round(orig_label_h / section_h, 6)
    label_coordonates = f"{label_number} {norm_new_label_x} {norm_new_label_y} {norm_orig_label_w} {norm_orig_label_h}"
    # see if label is fully visible in width and height and is not cut off
    is_visible_label = (
        ((orig_label_w / 2) + new_label_x)
        <= section_w  # 70% of the label is inside the width box
        and ((orig_label_h / 2) + new_label_y)
        <= section_h  # 70% of the label is inside the height box
        and norm_new_label_x >= 0
        and norm_new_label_y >= 0
        and norm_orig_label_w >= 0
        and norm_orig_label_h >= 0
    )
    # return label coordonates if the label is fully visible and empty string if not
    return label_coordonates if is_visible_label else ""


def generate_windows(input_folder, dest_path, window_wh=(640, 640), stride_percent=0.5):
    # load all images in a folder
    files = os.listdir(input_folder + "/images")
    window_width, window_height = window_wh
    stride = int(window_width * stride_percent)
    image_files = [f for f in files if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    count = 0
    # cycle trough images
    for image_name in image_files:
        # image object
        image = Image.open(f"{input_folder}/images/{image_name}")
        [img_name, img_extension] = image_name.rsplit(".", 1)
        image_width, image_height = image.size

        w_windows = int(image_width / stride)  # nr of strides in image width
        h_windows = int(image_height / stride)  # nr of strides in image height
        count = 0  # incrementing to name the files uniquely
        for y in range(h_windows):

            is_last_y_window = False
            # detecting the last y iteration
            if y + 1 == h_windows:
                # setting a variable to know it is the last iteration
                is_last_y_window = True
                # y position window logic
            new_slice_y = (
                image_height - window_height  # last window to the right
                if is_last_y_window == True  # if it's the last itaration
                else y * stride  # normal stride calculation for y
            )
            # cycling trough sections in height
            for x in range(w_windows):
                #
                # if this is the last horizontal window w_section iteration
                #
                is_last_x_window = False
                if x + 1 == w_windows:
                    is_last_x_window = True
                    # breakpoint()
                new_slice_x = (
                    int(image_width - window_width)
                    if is_last_x_window == True
                    else x * stride
                )
                # crop window to be
                new_slice_xy = (new_slice_x, new_slice_y)
                new_slice_wh = (window_width, window_height)
                section_image = crop_image(
                    image,
                    xy=new_slice_xy,
                    wh=new_slice_wh,
                )

                # save image file in output folder with _count appended postfix name
                image_dest_file = (
                    f"{dest_path}images/{img_name}_{count}.{img_extension}"
                )
                # if is_last_x_window:
                #     print(image_dest_file)
                # save label destination path
                label_dest_file = f"{dest_path}/labels/{img_name}_{count}.txt"
                # test image with bounding box generationg (to be commented out when actually using)
                section_image.save(image_dest_file)
                # read current image label
                # Open the file in read mode
                with open(f"{input_folder}/labels/{img_name}.txt", "r") as file:
                    # Read all lines into a list
                    lines = file.readlines()

                # Strip newline characters from each line (optional)
                labels = [line.strip() for line in lines]
                # cycle trough labels
                index = 0
                final_label_text = ""
                for label_data_string in labels:
                    label_data_list = label_data_string.split(" ")
                    label_wh = (
                        (float(label_data_list[3]) * image_width),
                        (float(label_data_list[4]) * image_height),
                    )
                    label_w, label_h = label_wh
                    label_xy = (
                        (float(label_data_list[1]) * image_width) + (label_w / 2),
                        (float(label_data_list[2]) * image_height) + (label_h / 2),
                    )

                    # get the label that's possibly inside this window
                    label_in_window = get_labelIn_window(
                        section_xy=(new_slice_x, new_slice_y),
                        section_wh=window_wh,
                        orig_label_xy=label_xy,
                        orig_label_wh=label_wh,
                        label_number=index,
                    )

                    if len(label_in_window) > 0:
                        final_label_text += label_in_window + "\n"
                    else:
                        final_label_text = label_in_window

                # save the label for the image:
                with open(label_dest_file, "w") as file:
                    file.write(final_label_text)
                count += 1

In [ ]:
from ultralytics import YOLO
from PIL import Image, ImageDraw
import os

# Load a pretrained model
#model = YOLO("./kfold_demo/fold_1/weights/best.pt")
#model = YOLO("./kfold_demo/fold_2/weights/best.pt")
model = YOLO("./kfold_demo/fold_3/weights/best.pt")


input_files = list_image_files("./datasetV3/test/images")

for file in input_files:
    # the width and height of the windows that will be cropped from the main image
    window_width, window_height = (640, 640)
    # the current image that is loaded from the list and to be processed
    main_image = Image.open(file)
    # get original image width and height
    image_width, image_height = main_image.width, main_image.height
    # calculate x and y stride
    stride_x, stride_y = (int(window_width / 2), int(window_height / 2))

    #
    # get number of windows on x and y axis's
    #
    # number of horizontal windows
    windows_x = int((image_width) / stride_x)
    # number of vertical windows
    windows_y = int((image_height) / stride_y)
    # * loop trough vertical windows:
    bounding_boxes = []
    box_found = False
    for window_y in range(windows_y):
        total_stride_y = window_y * stride_y
        for window_x in range(windows_x):
            total_stride_x = window_x * stride_x
            section_image = crop_image(
                main_image,
                xy=(total_stride_x, total_stride_y),
                wh=(window_width, window_height),
            )
            outputs = model.predict(
                save=False,
                source=section_image,
                show_labels=True,
                show_boxes=True
            )
            for out in outputs:
                if out.boxes.shape[0] == 0 or box_found == True:
                    continue
                prediction = out.boxes.xywhn[0]  # extract the
                prediction = prediction.tolist()  # convert to list
                prediction.insert(0, 1)  # prefix with the label 0
                box = get_box_coordonates(
                    section_xy=(total_stride_x, total_stride_y),
                    section_wh=(section_image.width, section_image.height),
                    box_xywhn=prediction,
                )
                bounding_boxes.append(box)
                box_found = True
    for box in bounding_boxes:
        draw = None
        image_for_label = main_image.copy()

        image = draw_rectangle(
            image=image_for_label, label_contents=box, return_image=True
        )
        file_name = os.path.basename(file)
        #image.save(f"./waldo_test_split1/{file_name}")
        #image.save(f"./waldo_test_split2/{file_name}")
        image.save(f"./waldo_test_split3/{file_name}")


0: 640x640 (no detections), 9.2ms
Speed: 1.6ms preprocess, 9.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.5ms
Speed: 1.4ms preprocess, 9.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.1ms
Speed: 1.4ms preprocess, 9.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.9ms
Speed: 1.5ms preprocess, 9.9ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.8ms
Speed: 1.4ms preprocess, 8.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.7ms
Speed: 1.4ms preprocess, 8.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.7ms
Speed: 1.3ms preprocess, 8.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.6ms
Speed: 1.4ms preprocess, 8.6ms inference, 0.7ms 

## Inference

Go to waldo_test_split(x) folder to see test results.
in our case model for split 2 is best according to F1 performance metrics - not underfit or overfit.